In [1]:
import os

mlflow_tracking_uri = os.getenv("MLFLOW_TRACKING_URI")

In [2]:
os.chdir("../")
%pwd

'/Users/vince/code/projects/aws-mlflow-mlops'

In [3]:
# Config entity
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [4]:
# Config manager
from src.aws_mlflow_mlops.constants import *
from src.aws_mlflow_mlops.utils.common import read_yaml, create_directories, save_json

# from src.aws_mlflow_mlops.entity.config_entity import ModelEvaluationConfig
from typing import Dict


class ConfigurationManager:
    """
    The ConfigurationManager class is responsible for managing the configuration of the data ingestion process.
    It reads configuration, parameters, and schema from YAML files, and creates necessary directories.

    Attributes:
        config (dict): Configuration read from a YAML file.
        params (dict): Parameters read from a YAML file.
        schema (dict): Schema read from a YAML file.
    """

    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
        schema_filepath: Path = SCHEMA_FILE_PATH,
    ):
        """
        The constructor for ConfigurationManager class.

        Parameters:
            config_filepath (str): The path to the configuration YAML file.
            params_filepath (str): The path to the parameters YAML file.
            schema_filepath (str): The path to the schema YAML file.
        """
        self.config: Dict = read_yaml(config_filepath)
        self.params: Dict = read_yaml(params_filepath)
        self.schema: Dict = read_yaml(schema_filepath)

        create_directories([self.config["artifacts_root"]])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config["model_evaluation"]
        params = self.params["ElasticNet"]
        schema = self.schema["TARGET_COLUMN"]

        create_directories([config["root_dir"]])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config["root_dir"],
            test_data_path=config["test_data_path"],
            model_path=config["model_path"],
            all_params=params,
            metric_file_name=config["metric_file_name"],
            target_column=schema["name"],
            mlflow_uri=mlflow_tracking_uri,
        )

        return model_evaluation_config

In [5]:
# Components
# Components
import os
from urllib.parse import urlparse

import joblib
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluation_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        # Load test data and model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_registry_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.evaluation_metrics(test_y, predicted_qualities)

            # Save metrics as local
            scores = {
                "rmse": rmse,
                "r2": r2,
                "mae": mae,
            }
            print(scores)
            save_json(path=Path(self.config.metric_file_name), data=scores)

            print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (model.alpha, model.l1_ratio))
            print("  RMSE: %s" % rmse)
            print("  MAE: %s" % mae)
            print("  R2: %s" % r2)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [6]:
# Pipeline
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2023-11-28 18:43:16,769: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-11-28 18:43:16,773: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-28 18:43:16,775: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-11-28 18:43:16,776: INFO: common: created directory at: artifacts]
[2023-11-28 18:43:16,777: INFO: common: created directory at: artifacts/model_evaluation]
{'rmse': 0.8050863675052666, 'r2': 0.008174066315615902, 'mae': 0.6594651869732101}
[2023-11-28 18:43:17,666: INFO: common: JSON file saved at: artifacts/model_evaluation/metrics.json]
Elasticnet model (alpha=0.900000, l1_ratio=0.800000):
  RMSE: 0.8050863675052666
  MAE: 0.6594651869732101
  R2: 0.008174066315615902


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2023/11/28 18:43:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 9
Created version '9' of model 'ElasticnetModel'.
